In [ ]:
#default_exp lightning.callbacks

In [ ]:
#export
import sys
import time
import datetime
from tqdm import tqdm
import wandb

import torch
import pytorch_lightning as pl
from pytorch_lightning import _logger as log

from src.lightning.core import *
from src.core import *

In [ ]:
#export
class WandbImageClassificationCallback(pl.Callback):
    """ Custom callback to add some extra functionalites to the wandb logger """
    
    def __init__(self, 
                 dm: pl.LightningDataModule, 
                 default_config: dict = None, 
                 num_batches:int = 16, 
                 log_train_batch: bool = False,
                 log_preds: bool = False,
                 log_conf_mat: bool = True,):
        
        # class names for the confusion matrix
        self.class_names = list(conf_mat_idx2lbl.values())
        
        # counter to log training batch images
        self.dm = dm
        self.num_bs = num_batches
        self.curr_epoch = 0
        self.log_train_batch = log_train_batch
        self.log_preds = log_preds
        self.val_imgs, self.val_labels = None, None
        self.log_conf_mat = log_conf_mat
        self.default_config = default_config
        
    def on_train_start(self, trainer, pl_module, *args, **kwargs):
        try:
            # log model to the wandb experiment
            wandb.watch(models=pl_module.model, criterion=pl_module.loss_func)
        except:
            log.info("Skipping wandb.watch --->")
            
        train_augs, valid_augs = self.dm.train_augs, self.dm.valid_augs
        self.train_config = dict(train_augments=train_augs, valid_augments=valid_augs)
        
        if self.default_config is not None: 
            try:
                wandb.config.update(self.default_config)
                wandb.config.update(self.train_config)
                log.info("wandb config updated -->")
            except: 
                log.info("Skipping update wandb config -->")
        
    def on_train_epoch_end(self, trainer, pl_module, *args, **kwargs):
        if self.log_train_batch:
            if pl_module.one_batch_of_image is None:
                log.info(f"{self.config_defaults['mixmethod']} samples not available . Skipping --->")
                pass

            else:
                one_batch = pl_module.one_batch_of_image[:self.num_bs]
                train_ims = one_batch.data.to('cpu')
                trainer.logger.experiment.log({"train_batch":[wandb.Image(x) for x in train_ims]}, commit=False)
        
    def on_validation_epoch_end(self, trainer, pl_module, *args, **kwargs):
        if self.log_preds:
            if self.val_imgs is None and self.val_labels is None:
                self.val_imgs, self.val_labels = next(iter(self.dm.val_dataloader()))
                self.val_imgs, self.val_labels = self.val_imgs[:self.num_bs], self.val_labels[:self.num_bs]
                self.val_imgs = self.val_imgs.to(device=pl_module.device)

            logits = pl_module(self.val_imgs)
            preds  = torch.argmax(logits, 1)
            preds  = preds.data.cpu()
            trainer.logger.experiment.log({"predictions": [wandb.Image(x, caption=f"Pred:{pred}, Label:{y}") for x, pred, y in zip(self.val_imgs, preds, self.val_labels)]},
                                          commit=False)
            
    def on_epoch_start(self, trainer, pl_module, *args, **kwargs):
        pl_module.val_labels_list = []
        pl_module.val_preds_list  = []
    
    def on_epoch_end(self, trainer, pl_module, *args, **kwargs):
        if self.log_conf_mat:
            val_preds  = torch.tensor(pl_module.val_preds_list).data.cpu().numpy()
            val_labels = torch.tensor(pl_module.val_labels_list).data.cpu().numpy()
            log_dict = {'conf_mat': wandb.plot.confusion_matrix(val_preds,val_labels,self.class_names)}
            wandb.log(log_dict,commit=False)

In [ ]:
#export
class LitProgressBar(pl.callbacks.ProgressBar):
    "Custom Progressbar callback for Lightning Training"
    def init_sanity_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for the validation sanity run. """
        bar = tqdm(
            desc='Validation sanity check',
            position=(2 * self.process_position),
            disable=self.is_disabled,
            leave=False,
            dynamic_ncols=True,
            file=sys.stdout,
        )
        return bar
    
    def init_train_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for training. """
        bar = tqdm(
            desc='Training',
            initial=self.train_batch_idx,
            position=(2 * self.process_position),
            disable=self.is_disabled,
            leave=False,
            dynamic_ncols=True,
            file=sys.stdout,
            smoothing=0,
        )
        return bar
    
    def init_validation_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for validation. """
        bar = tqdm(
            desc='Validating',
            position=(2 * self.process_position + 1),
            disable=True,
            leave=False,
            dynamic_ncols=True,
            file=sys.stdout
        )
        return bar
    
    def init_test_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for testing. """
        bar = tqdm(
            desc='Testing',
            position=(2 * self.process_position),
            disable=self.is_disabled,
            leave=False,
            dynamic_ncols=True,
            file=sys.stdout
        )
        return bar

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from torch import nn
from src.networks import *

In [ ]:
train_augs = A.Compose([
    A.RandomResizedCrop(224, 224, p=1.0),
    A.RandomBrightness(limit=0.1),
    A.HueSaturationValue(20, 20, 20),
    A.HorizontalFlip(),
    A.Normalize(p=1.0),
    ToTensorV2(p=1.0)])

valid_augs = A.Compose([
    A.Resize(224, 224, p=1.0),
    A.Normalize(p=1.0),
    ToTensorV2(p=1.0)])

csv = "../../leaf-disease-classification-kaggle/data/stratified-data-5folds.csv"
ims = "../../Datasets/cassava/train_images/"
dm = CassavaLightningDataModule(csv, ims, curr_fold=0, train_augs=train_augs, valid_augs=valid_augs, bs=32, num_workers=0)

example_conf = dict(
    mixmethod = None,
    loss_function = dict(_target_='src.core.LabelSmoothingCrossEntropy', eps=0.1),
    learning_rate = 1e-03,
    lr_mult = 100,
    optimizer = dict(_target_='src.opts.Ranger', betas=(0.9, 0.99), eps=1e-06, weight_decay=0),
    scheduler = dict(_target_='src.opts.FlatCos', num_epochs=100),
    metric_to_track = None,
    step_after = "step",
    frequency = 1,
)


encoder = timm.create_model('resnet18', pretrained=False)
model = TransferLearningModel(encoder, cut=-2, c=5, act=nn.ReLU(inplace=True))
model = LightningCassava(model=model, conf=example_conf)

Loss Function : LabelSmoothingCrossEntropy()
12/31/2020 21:05:33 - INFO - lightning -   Loss Function : LabelSmoothingCrossEntropy()


In [ ]:
#export
class PrintLogsCallback(pl.Callback):
    "Logs Training logs to console after every epoch"
    def __init__(self, print_str: str = None, logger=None):
        self.print_str = 'Epoch: [{}] eta: {} loss: {:.4f} acc: {:.4f} valid_loss: {:.4f} valid_acc: {:.4f}'
        
        if logger is None: self.logger = log
        else             : self.logger = logger
    
    def on_epoch_start(self, *args, **kwargs):
        self.eta_start = time.time()
    
    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        train_loss = metrics['train/loss']
        train_acc  = metrics['train/acc']
        valid_loss = metrics['valid/loss']
        valid_acc  = metrics['valid/acc']
        
        end_time = time.time()
        self.eta_string = str(datetime.timedelta(seconds=int(end_time-self.eta_start)))
        self.curr_epoch = int(trainer.current_epoch)
        print_str = self.print_str.format(self.curr_epoch, self.eta_string, 
                                        train_loss, train_acc, 
                                        valid_loss, valid_acc)
        self.logger.info(print_str)
    
    def on_test_epoch_end(self, trainer, pl_module, *args, **kwargs):
        metrics = trainer.callback_metrics
        train_loss = metrics['train/loss']
        train_acc  = metrics['train/acc']
        valid_loss = metrics['valid/loss']
        valid_acc  = metrics['valid/acc']
        test_loss  = metrics['test/loss']
        test_acc   = metrics['test/acc']
        
        
        fmt_str1 = "Summary: [Train] loss: {:.4f} acc: {:.4f}"
        fmt_str2 = "Summary: [Valid] loss: {:.4f} acc: {:.4f}"
        fmt_str3 = "Summary: [Test]  loss: {:.4f} acc: {:.4f}"
        
        str1 = fmt_str1.format(train_loss, train_acc)
        str2 = fmt_str2.format(valid_loss, valid_acc)
        str3 = fmt_str3.format(test_loss, test_acc)
        
        self.logger.info(str1)
        self.logger.info(str2)
        self.logger.info(str3)

In [ ]:
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)
logger = logging.getLogger("metrics")

In [ ]:
trainer = pl.Trainer(callbacks=[LitProgressBar(), PrintLogsCallback(logger=logger)], 
                     num_sanity_val_steps=0, max_epochs=2, limit_train_batches=1, 
                     limit_val_batches=1, limit_test_batches=1, weights_summary=None)

GPU available: False, used: False
12/31/2020 21:08:39 - INFO - lightning -   GPU available: False, used: False
TPU available: False, using: 0 TPU cores
12/31/2020 21:08:39 - INFO - lightning -   TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, datamodule=dm)

Optimizer: Ranger  LR's: (1e-05, 0.001)
12/31/2020 21:08:40 - INFO - lightning -   Optimizer: Ranger  LR's: (1e-05, 0.001)
LR Scheculer: FlatCos
12/31/2020 21:08:40 - INFO - lightning -   LR Scheculer: FlatCos


Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
Epoch 0: 100%|██████████| 2/2 [00:08<00:00,  4.22s/it, loss=1.700, v_num=21, valid/loss=1.64, valid/acc=0.125]

12/31/2020 21:08:48 - INFO - metrics -   Epoch: [0] eta: 0:00:08 loss: 1.6997 acc: 0.1875 valid_loss: 1.6374 valid_acc: 0.1250


Epoch 1: 100%|██████████| 2/2 [00:08<00:00,  4.21s/it, loss=1.763, v_num=21, valid/loss=1.64, valid/acc=0.125]

12/31/2020 21:08:57 - INFO - metrics -   Epoch: [1] eta: 0:00:08 loss: 1.8265 acc: 0.0938 valid_loss: 1.6421 valid_acc: 0.1250


1

In [ ]:
_ = trainer.test(model, datamodule=dm, verbose=False)

Testing: 100%|██████████| 1/1 [00:02<00:00,  2.12s/it]

12/31/2020 21:09:00 - INFO - metrics -   Summary: [Train] loss: 1.8265 acc: 0.0938
12/31/2020 21:09:00 - INFO - metrics -   Summary: [Valid] loss: 1.6421 acc: 0.1250
12/31/2020 21:09:00 - INFO - metrics -   Summary: [Test]  loss: 1.6421 acc: 0.1250


In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 00a_lightning.core.ipynb.
Converted 00b_fastai.core.ipynb.
Converted 01_mixmethods.ipynb.
Converted 01a_lightning.callbacks.ipynb.
Converted 02_layers.ipynb.
Converted 02a_networks.ipynb.
Converted 03_optimizers.ipynb.
Converted index.ipynb.
